# Regression: I

*S. R. Taylor (2021)*

This lecture and notebook are based on the "Regression" and "Regression2" lectures of of G. Richards' "Astrostatistics" class at Drexel University (PHYS 440/540, https://github.com/gtrichards/PHYS_440_540), which in turn are based on materials from Andy Connolly, and Ivezic et al. Chapter 8.

##### Reading:

- [Textbook](http://press.princeton.edu/titles/10159.html) Chapter 8.

---

## Contents
* [What is regression?](#one)
* [2-D linear regression](#two)
* [Multivariate linear regression](#three)
* [Polynomial regression](#four)
* [Basis function regression](#five)
* [Kernel regression](#six)
* [Over/under-fitting](#seven)
* [Cross-validation](#eight)
    
---

***Exercises required for class participation are in <font color='red'>red</font>.***

---

## What is regression? <a class="anchor" id="one"></a>

[**Regression**](https://en.wikipedia.org/wiki/Regression_analysis) is about determining the relationship between an independent variable, $x$, and the variable that depends on it, $y$, where the expectation value of $y$ is $E[y|x]$. Crudely speaking, it is the set of techniques associated with curve fitting. In contrast to density estimation, clustering, and dimensional reduction (which was largely *unsupervised*) **regression is a *supervised* process**.

- Generally, we'd like to infer the true generating pdf of observations from a multi-dimensional sample of data that is drawn from that pdf, using parametric or non-parametric models to do so.  
- This is hard, so ***regression seeks to determine the expectation value of $y$ (given $x$, i.e. the conditional expectation value) rather than the full pdf.***  
- That is, for a given value of $x$ you'd like to predict the pdf of $y$, but you are going to settle for a **point estimate** (single value).

When I say "regression", you probably think about linear least-squares fitting (fitting a line) or maximum likelihood analysis. However, adopting a Bayesian perspective enables a more physical intuition that includes how we can do regression in the case of both errors and limits on the data. 

Let's start by looking at the classic example of fitting a straight line to some data points in 2-D as illustrated by Ivezic, Figure 8.1:

![Ivezic, Figure 8.1a](http://www.astroml.org/_images/fig_linreg_inline_1.png)

- Here we have 4 data points $\{x_1,x_2,x_3,x_4\}$ drawn from $y=\theta_1 x + \theta_0$, where $\theta_1 = 1$ and $\theta_0 = 0$.  
- Each data point provides a joint constraint on $(\theta_0,\theta_1)$.  
- If there were no uncertainties in the measurement of each $y$, then each new point would yield a straight line constraint in $(\theta_0,\theta_1)$ of 
$\theta_0 = y_i - \theta_1 x_i$.  
- Think about the constraints from the first point. You could fit an infinite number of different lines through it, all of which would completely cover the $(x,y)$ plane. ***However the parameters of those infinite lines are constrained by $x_1$ to lie along a unique line in the $(\theta_0,\theta_1)$ plane.*** See the first panel in the plot from the next cell.

In [ ]:
# Execute this cell
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
# Ivezic, Figure 8.1, modified by SRT and GTR
# Author: Jake VanderPlas
# License: BSD
#   The figure produced by this code is published in the textbook
#   "Statistics, Data Mining, and Machine Learning in Astronomy" (2013)
#   For more information, see http://astroML.github.com
#   To report a bug or issue, use the following forum:
#    https://groups.google.com/forum/#!forum/astroml-general
from astroML.plotting.mcmc import convert_to_stdev

#------------------------------------------------------------
# Set up the data and errors
np.random.seed(13)
a = 1
b = 0

x = np.array([-1, 0.44, -0.16, 1.0])
y = a * x + b
dy = np.array([0.01, 0.01, 0.01, 0.01])

y = np.random.normal(y, dy)

# add a fourth point which is a lower bound
x5 = 1.0
y5 = a * x5 + b + 0.0 

#------------------------------------------------------------
# Compute the likelihoods for each point
a_range = np.linspace(0, 2, 80)
b_range = np.linspace(-1, 1, 80)
logL = -((a_range[:, None, None] * x + 
          b_range[None, :, None] - y) / dy) ** 2
sigma = [convert_to_stdev(logL[:, :, i]) for i in range(4)]

# compute best-fit from first three points
logL_together = logL.sum(-1)
i, j = np.where(logL_together == np.max(logL_together))

amax = a_range[i[0]]
bmax = b_range[j[0]]

#------------------------------------------------------------
# Plot the first figure: the points and errorbars
fig1 = plt.figure(figsize=(6, 4))
ax1 = fig1.add_subplot(111)

# Draw the true and best-fit lines
xfit = np.array([-1.5, 1.5])
ax1.plot(xfit, a * xfit + b, ':k', label='True fit')
ax1.plot(xfit, amax * xfit + bmax, '--k', 
         label='fit to $\{x_1, x_2, x_3\}$')

ax1.legend(loc=2)

ax1.errorbar(x, y, dy, fmt='ok')
ax1.errorbar([x5], [y5], [[0.5], [0]], 
             fmt='_k', uplims=True)

for i in range(4):
    ax1.text(x[i] + 0.05, y[i] - 0.3, 
             "$x_{%i}$" % (i + 1))
ax1.text(x5 + 0.05, y5 - 0.5, "$x_4$")

ax1.set_xlabel('$x$')
ax1.set_ylabel('$y$')

ax1.set_xlim(-1.5, 1.5)
ax1.set_ylim(-2, 2)

#------------------------------------------------------------
# Plot the second figure: likelihoods for each point
fig2 = plt.figure(figsize=(10, 10))
fig2.subplots_adjust(hspace=0.05, wspace=0.05)

# plot likelihood contours
for i in range(5):
    ax = fig2.add_subplot(321 + i)
    for j in range(min(i + 1, 4)):
        ax.contourf(a_range, b_range, sigma[j].T,
                    levels=(0, 0.683, 0.955, 0.997),
                    cmap=plt.cm.binary, alpha=0.5)

# plot the excluded area from the fourth point
axpb = a_range[:, None] * x5 + b_range[None, :]
mask = y5 < axpb
fig2.axes[4].fill_between(a_range, y5 - x5 * a_range, 
                          2, color='k', alpha=0.5)

# Label and adjust axes
for i in range(5):
    ax = fig2.axes[i]

    ax.text(1.98, -0.98, "$x_{%i}$" % (i + 1), 
            ha='right', va='bottom')

    ax.plot([0, 2], [0, 0], ':k', lw=1)
    ax.plot([1, 1], [-1, 1], ':k', lw=1)

    ax.set_xlim(0.001, 2)
    ax.set_ylim(-0.999, 1)

    if i in (1, 3):
        ax.yaxis.set_major_formatter(plt.NullFormatter())
    if i in (0, 1, 2):
        ax.xaxis.set_major_formatter(plt.NullFormatter())
    if i in (0, 2):
        ax.set_ylabel(r'$\theta_0$')
    if i in (3, 4):
        ax.set_xlabel(r'$\theta_1$')

plt.show()

- Now look at the top right panel. The addition of another data point is sufficient to fully constrain the properties of a linear relationship, shown by the intersection of the $(\theta_0,\theta_1)$ lines from each point.
- More data points yield more constraints, and the best fit solution is the intersection of all lines in model parameter space (as shown in the next 3 panels).

**NOTE:** The errors are really small so that the constraints are just lines in parameter space. The 4th $x$ value is shown as a point estimate and a limit, so that you can see that the excluded region from the limit just follows the line that would result had it been a detection.

If the measurements have uncertainties then the lines are linear-shaped distributions as illustrated in the actual Figure 8.1 from Ivezic as shown below.

![Ivezic, Figure 8.1b](http://www.astroml.org/_images/fig_linreg_inline_2.png)

### Bayesian perspective on regression

If we take a Bayesian approach to regression, then we can easily write the posterior pdf for the model parameters as

$$p(\theta|\{x_i, y_i\},I) \propto p(\{x_i,y_i\} | \theta, I) \, p(\theta, I),$$

where the data likelihood is the product over the individual event likelihoods, which can be written as

$$p(y_i|x_i,{\theta}, I) = e(y_i|y)$$

with $e(y_i|y)$ being the probability of getting $y_i$ given the adopted model of $y=f(x|\theta)$ (i.e. the error distribution). If the error distribution is Gaussian then,

$$p(y_i|x_i,{\theta}, I) = {1 \over \sigma_i \sqrt{2\pi}} \, \exp{\left({-[y_i-f(x_i|{\theta})]^2 \over 2 \sigma_i^2}\right)}.$$

## 2-D Linear Regression <a class="anchor" id="two"></a>

Let's start with the simplest case: a linear model with independent variable, $x$, and dependent variable, $y$:

$$y_i = \theta_0 + \theta_1 x_i + \epsilon_i,$$

where $\theta_0$ and $\theta_1$ are the coefficients of the model that we are trying to estimate, and $\epsilon_i$ is an additive noise term with $\epsilon_i = \mathscr{N}(0,\sigma_i)$. 

As we've seen many times, the full data likelihood is the product of each individual event likelihood:

$$p(\{y_i\}|\{x_i\},{\theta}, I) \propto \prod_{i=1}^N \exp \left(\frac{-(y_i- (\theta_0 + \theta_1x_{i}))^2}{  2\sigma_i^2}\right).$$

Assuming a flat/uninformative prior, the log of the posterior probability is then

$$\ln(p({\theta}|\{x_i, y_i\},I)) \propto \ln \mathcal(L)  \propto \sum_{i=1}^N \left(\frac{-(y_i- (\theta_0 + \theta_1x_{i}))^2}{  2\sigma_i^2}\right).$$

***We want to find the values of $\theta$ that maximize this expression, which is the same as minimizing the least squares.*** 

### Homoscedastic uncertainty scenario

With uncertainties that are the same for all points, this minimization yields

$$\theta_1 = \frac{\sum_i^N x_i y_i - \bar{x}\bar{y}}{\sum_i^N(x_i-\overline{x})^2},$$

and

$$\theta_0 = \overline{y} - \theta_1\overline{x},$$

where $\overline{x}$ and $\overline{y}$ are the mean values.

The estimate of the variance and the standard errors of the estimated parameters are

$$\sigma^2 = \sum_{i=1}^N (y_i - \theta_0 + \theta_1 x_i)^2,$$

$$\sigma_{\theta_1}^2 = \sigma^2\frac{1}{\sum_i^N(x_i-\overline{x})^2},$$

$$\sigma_{\theta_0}^2 = \sigma^2\left(\frac{1}{N} + \frac{\overline{x}^2}{\sum_i^N(x_i-\overline{x})^2}\right).$$

### Heteroscedastic uncertainty scenario

If the errors are instead different for each point, it is better to think of the problem in matrix notation:

$$Y = M \theta$$

where $Y$ is an $N$-dimensional vector of values ${y_i}$,

$$Y = \left[
    \begin{array}{c}
    y_0\\
    \vdots\\
    y_{N-1}
    \end{array}
    \right].
$$

For the straight line model, $\theta$ is simply a two-dimensional vector of regression coefficients,

$$
\theta = \left[
            \begin{array}{c}
            \theta_0\\
            \theta_1
            \end{array}
          \right],
$$

and **$M$ is a called the design matrix**

$$
M = \left[
        \begin{array}{cc}
        1 & x_0\\
        \vdots & \vdots\\
        1 & x_{N-1}
        \end{array}
    \right],
$$

where the constant in the first column of $M$ captures the zeropoint (i.e. the constant $y$-intercept) in the regression.

The log-likelihood can then be written as

$$ \ln\mathcal(L) = -\frac{1}{2}(Y-M\theta)^T C^{-1} (Y-M\theta) - \frac{1}{2}\ln[\mathrm{det}(C)]$$

where we encapsulate uncertainties in the ($N\times N$) covariance matrix

$$C=\left[
        \begin{array}{cccc}
        \sigma_{0}^2 & 0 & \cdots & 0 \\
        \vdots & \vdots & \cdots & \vdots \\
        0 & 0 & \cdots & \sigma_{N-1}^2 \\
        \end{array}
    \right].
$$

and the maximum likelihood solution for the regression can be analytically solved and expressed as

$$\theta = (M^T C^{-1} M)^{-1} (M^T C^{-1} Y),$$

which minimizes the sum of squares, and gives uncertainties on $\theta$ of 

$$\Sigma_\theta = \left[
                    \begin{array}{cc}
                    \sigma_{\theta_0}^2 & \sigma_{\theta_0\theta_1} \\
                    \sigma_{\theta_0\theta_1} & \sigma_{\theta_1}^2
                    \end{array}
                  \right]
                    = [M^T C^{-1} M]^{-1}.
$$

With `numpy` it is straightforward to write the matrices as multi-dimensional arrays, and do the linear algebra (provided the matrices are invertible), then calculate the regression coefficients.

Let's start with some noisy data and the known solution. <font color='red'>Write the equation to predict the $y$ values and execute this cell.</font>

In [ ]:
theta0 = 95 # Known value of b (theta0)
theta1 = 10 # Known value of m (theta1)
noise_std = 1.0 
noise = np.random.randn(100, 1) * noise_std
dy = noise

X = np.random.rand(100, 1)
y = theta0 + theta1 * X + dy

X_grid = np.linspace(0,1,11)
y_true = ____ + ____*____ # Write equation needed to plot a line based on known truth

plt.plot(X, y, "b.", label='Measured')
plt.plot(X_grid, y_true, "k-", label='Actual')
plt.xlabel("$x$", fontsize=14)
plt.ylabel("$y$", rotation=0, fontsize=14)
plt.legend()
plt.show()

<font color='red'>Now solve for the $\theta$ values. First using matrix math as shown below.</font>

In [ ]:
# diagonal noise matrix
C = np.identity(len(X[:,0])) * noise_std**2

# design matrix
M = np.column_stack((np.ones(len(X[:,0])), X[:,0]))

# Sigma^-1 = M^T * C^-1 * M
A = np.dot(np.dot(M.T, np.linalg.pinv(C)), M)

# M^T * C^-1 * y
B = np.dot(np.dot(M.T, np.linalg.pinv(C)), y)

# theta = Sigma * M^T * C^-1 * y
theta = np.dot(np.linalg.pinv(A),B)
print(theta)

<font color='red'>Use the results to make predictions and compare to the known answer.</font>

In [ ]:
X_new = np.array([[0], [1]]) #like X_grid, but just with the endpoints
y_pred = ____ + ____*____  # Complete 

plt.plot(X_new, ____, "r--", linewidth=2, label="Predictions")
plt.plot(X_grid, y_true, "k-.", label="Actual") # Comment this out to see that it agrees with the above
plt.plot(X, y, "b.", label='Measured')
plt.xlabel("$x$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.legend(loc="upper left", fontsize=14)
plt.show()

#N.B. you could also do this as 
#y_pred = np.dot(MM,theta)
#if MM were defined appropriately
#MM = np.column_stack((np.ones(len(X_grid)),X_grid))
#y_pred = np.dot(MM,theta)
#plt.plot(MM[:,1], y_pred, "r--", linewidth=2, label="Predictions")

<font color='red'>Now do it with [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) from `Scikit-Learn`.</font>


In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X, y, sample_weight=1.0)

theta0 = lin_reg.intercept_
theta1 = lin_reg.coef_

print(theta0, theta1)
y_pred2 = lin_reg.predict(X_new)

<font color='red'>Print $C$, $M$, $A$, $B$, and $\theta$ and make sure that you understand how these are constructed.</font>

In [ ]:
print(C)
print(M)
print(A)
print(B)
print(theta)

Now compare `y_pred` to `y_pred2`.

In [ ]:
print(y_pred)
print(y_pred2)

So, what we did was to replace
```
C = np.identity(len(X))*(dy*dy)
M = np.column_stack((np.ones(len(X)),X))
A = np.dot(np.dot(M.transpose(),np.linalg.pinv(C)),M)
B = np.dot(np.dot(M.transpose(),np.linalg.pinv(C)),y)
theta = np.dot(np.linalg.pinv(A),B)

y_pred = np.dot(M_new,theta) # using design matrix at new x locations
```

with
```
LRmodel = LinearRegression()
LRmodel.fit(X, y, sample_weight=noise_std)
y_pred = LRmodel.predict(X_new)
```

Below is another example from the first panel of Figure 8.2, where we have done a straight-line regression to data with a non-linear correlation.  

***These are redshifts and distance moduli for supernovae. Calibrating their relationship gives us the parameters that describe things like the expansion rate of the Universe (Hubble constant) and the fractional composition of baryons, dark energy, etc.***

A reminder on nomenclature for `Scikit-Learn`: 
- $X$ is the multidimensional matrix of $N$ "objects", each with $K$ attributes.  
- $y$ is the dependent variable that represents the measured value for each of those $N$ objects.  
- What is new here is that are adding $dy$, which is the uncertainty on $y$. 

The fitting algorithms are going to be of the form:

```model.fit(X,y,dy)```.

In [ ]:
# Ivezic, Figure 8.2, modified by GTR (to just plot first panel)
# Author: Jake VanderPlas
# License: BSD
#   The figure produced by this code is published in the textbook
#   "Statistics, Data Mining, and Machine Learning in Astronomy" (2013)
#   For more information, see http://astroML.github.com
#   To report a bug or issue, use the following forum:
#    https://groups.google.com/forum/#!forum/astroml-general
from astropy.cosmology import LambdaCDM
from astroML.datasets import generate_mu_z
from astroML.linear_model import (LinearRegression, PolynomialRegression,
                                  BasisFunctionRegression, NadarayaWatson)

#------------------------------------------------------------
# Generate data
z_sample, mu_sample, dmu = generate_mu_z(100, random_state=0)
X = z_sample[:,None]
y = mu_sample
dy = dmu

cosmo = LambdaCDM(H0=70, Om0=0.30, Ode0=0.70, Tcmb0=0)
z = np.linspace(0.01, 2, 1000)
y_true = cosmo.distmod(z)
mu_true = y_true

X_grid = z[:,None]

#------------------------------------------------------------
# Define our classifiers
basis_mu = np.linspace(0, 2, 15)[:, None]
basis_sigma = 3 * (basis_mu[1] - basis_mu[0])

subplots = [221, 222, 223, 224]
classifiers = [LinearRegression(),
               PolynomialRegression(4),
               BasisFunctionRegression('gaussian',
                                       mu=basis_mu, 
                                       sigma=basis_sigma),
               NadarayaWatson('gaussian', h=0.1)]
text = ['Straight-line Regression',
        '4th degree Polynomial\n Regression',
        'Gaussian Basis Function\n Regression',
        'Gaussian Kernel\n Regression']

# number of constraints of the model.  Because
# Nadaraya-watson is just a weighted mean, it has only one constraint
n_constraints = [2, 5, len(basis_mu) + 1, 1]

#------------------------------------------------------------
# Plot the results
fig = plt.figure(figsize=(5, 5))
fig.subplots_adjust(left=0.1, right=0.95,
                    bottom=0.1, top=0.95,
                    hspace=0.05, wspace=0.05)

ax = fig.add_subplot(111)
i=0 #Just show the straight-line regression

# fit the data
clf = classifiers[i]
clf.fit(X, y, dy)

mu_sample_fit = clf.predict(X)
mu_fit = clf.predict(X_grid)
mu_fit_lin = mu_fit #Save for later to compare

chi2_dof = (np.sum(((mu_sample_fit - mu_sample) / dmu) ** 2)
                / (len(mu_sample) - n_constraints[i]))

ax.plot(z, mu_fit, '-r')
ax.plot(z, mu_true, '--', c='gray')
ax.errorbar(z_sample, mu_sample, dmu, fmt='.k', ecolor='gray', lw=1)

ax.text(0.5, 0.05, r"$\chi^2_{\rm dof} = %.2f$" % chi2_dof,
            ha='center', va='bottom', transform=ax.transAxes)

ax.set_xlim(0.01, 1.8)
ax.set_ylim(36.01, 48)
ax.text(0.05, 0.95, text[i], ha='left', va='top',
            transform=ax.transAxes)

ax.set_ylabel(r'$\mu$')
ax.set_xlabel(r'$z$')

plt.show()

In this plot, the best-fit linear relationship is red, and the true generating relationship is dashed. 

### A Word of Caution

[Anscombe's Quartet](https://en.wikipedia.org/wiki/Anscombe%27s_quartet) is a demonstration of why you should always visualize your data and not just blindly use the parameters of some black-box fitting algorithm.

![Anscombe's Quartet](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Anscombe%27s_quartet_3.svg/1000px-Anscombe%27s_quartet_3.svg.png)

Each of these data sets has $11$ points and basic statistical properties that are identical. For example, they all have the same best fit when assuming a linear model.

## Multivariate linear regression <a class="anchor" id="three"></a>

In the above cases, we were doing 2-D linear regression with a univariate $X$.  If $X$ is instead multivariate, then we fit a hyperplane rather than a straight line

$$y_i =\theta_0 + \theta_1x_{i1} + \theta_2x_{i2} + \cdots +\theta_kx_{ik} + \epsilon_i.$$
 
The design matrix, $M$, is now 

$$M = \left(
        \begin{array}{ccccccc}
        1 & x_{01} & x_{02} & . & x_{0k}\\
        1 & x_{11} & x_{12} & . & x_{1k}\\
        . & . & . & .  & . \\
        1 & x_{N1} & x_{N2} & . & x_{Nk}\\
        \end{array}
      \right)
$$

but the whole formalism is exactly the same as before.

Scikit-Learn obviously has a [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) routine, but it does not explicitly account for heteroscedastic errors, so above we used the AstroML routine instead as illustrated below.

In [ ]:
from astroML.linear_model import LinearRegression

Xtest = np.random.random((100,2)) # 100 points in 2D
dytest = np.random.random(100) # heteroscedastic errors
ytest = np.random.normal(Xtest[:,0] + Xtest[:,1], dytest) # y = 0 + 1*x1 + 1*x2

model = LinearRegression()
model.fit(Xtest, ytest, dytest)

print(model.coef_)
#y_pred = model.predict(X)

## Polynomial Regression <a class="anchor" id="four"></a>

We introduced regression with examples of straight-line fitting, but we can think of it more generically in terms of **[polynomical regression](https://en.wikipedia.org/wiki/Polynomial_regression)** with $y=f(x|\theta)$ and

$$y_i =\theta_0 + \theta_1 x_{i} + \theta_2 x_{i}^2 + \theta_3 x_{i}^3 + \cdots.$$   

For example, maybe $y$ depends on the area or volume of an object, but you have mesured the length. More generally, this is like fitting a model that is a Taylor expansion of the exact $f(x)$ to determine amplitudes of the different polynomial terms.

For polynomical regression, the design matrix $M$ is now
 
$$
M = \left(
        \begin{array}{cccccc}
        1 & x_{0} & x_{0}^2 & x_{0}^3\\
        1 & x_{1} & x_{1}^2 & x_{1}^3\\
        . & . & . & . \\
        1 & x_{N} & x_{N}^2 & x_{N}^3\\
        \end{array}
    \right).
$$

**NOTE:** Be careful with terminology!! ***This is still Linear Regression since the model only depends linearly on the parameters.*** The "linear" in linear regression corresponds to these parameters, not the independent variable $x$.

As with linear regression, we'll use `PolynomialRegression` from `AstroML`.

In [ ]:
# Typical call
from astroML.linear_model import PolynomialRegression

Xtest = np.random.random((100,1))
ytest = 2.5*Xtest[:,0]**2 + 5.0*Xtest[:,0]**3

degree = 3
model = PolynomialRegression(degree) # fit 3rd degree polynomial
model.fit(Xtest, ytest)

y_pred = model.predict(Xtest)
n_constraints = degree + 1

print(model.coef_)

<font color='red'>Recreate the supernovae figure from above now using the `PolynomialRegression` algorithm with `degree=4`. *(Hint: Don't overthink it.)* </font>

In [ ]:
#fit data using standard package
degree = ___
n_constraints = degree + 1
poly = ___(___)
poly.fit(X, y, dy)
mu_fit = poly.predict(X_grid)
mu_sample_fit = poly.predict(X)

chi2_dof = (np.sum(((mu_sample_fit - mu_sample) / dmu)**2) / 
            (len(mu_sample) - n_constraints))

fig = plt.figure(figsize=(6, 6))

#plot the data
ax = fig.add_subplot(111)
ax.plot(z, mu_fit, '-k')
ax.plot(z, mu_true, '--', c='gray')
ax.errorbar(z_sample, mu_sample, dmu, 
            fmt='.k', ecolor='gray', lw=1)

ax.text(0.5, 0.05, r"$\chi^2_{\rm dof} = %.2f$" % chi2_dof,
    ha='center', va='bottom', 
        transform=ax.transAxes, fontsize=14)

ax.set_xlim(0.01, 1.8)
ax.set_ylim(36.01, 48)
ax.text(0.05, 0.95, 'Polynomial regression', ha='left', va='top',
                transform=ax.transAxes)

ax.set_ylabel(r'$\mu$')
ax.set_xlabel(r'$z$')
ax.plot(z, mu_fit_lin, '-r')
plt.show()

The best-fit $4$th degree polynomial is solid black, the linear fit from before is solid red, and the true generating relationship is dashed.

<font color='red'>Can you make the same code do straight-line regression?</font>

## Basis function regression <a class="anchor" id="five"></a>

If we consider a function in terms of a sum over bases (this can be polynomials, Gaussians, sines) then we can solve for the linear amplitude coefficients using regression. ***So regression with arbitrary basis terms that may be non-linear in $x$ is still linear regression, since the model is linear in the regression parameters.*** 

Above we have used polynomials, but we could substitute $x_{0}^2$ etc for Gaussians (where we fix $\sigma$ and $\mu$ and fit for the amplitude) as long as the attribute we are fitting for is linear. So if straight-line regression is just a special case of polynomial regression, then polynomial regression is just a special case of basis function regression. All of these are linear regression techniques.

In [ ]:
#Basis function regression looks like this
from astroML.linear_model import BasisFunctionRegression

# data
Xtest = np.random.random((100,1))
ytest = np.random.normal(Xtest[:,0], dy)

# mean positions of the 10 Gaussians in the model
X_gridtest = np.linspace(0,1,10)[:, None]
# widths of these Gaussians
sigma_test = 0.1

model = BasisFunctionRegression('gaussian', mu=X_gridtest, sigma=sigma_test)
model.fit(Xtest, ytest, dytest)

y_pred = model.predict(Xtest)
print(model.coef_)

We'll now repeat the supernova data example using basis function regression.

In [ ]:
#------------------------------------------------------------
# Define our Gaussians
nGaussians = 10
basis_mu = np.linspace(0,2,nGaussians)[:, None]
basis_sigma = 1.0 * (basis_mu[1] - basis_mu[0])

n_constraints = nGaussians+1

#fit data using gaussian-based basis function regression
bfr = BasisFunctionRegression('gaussian', mu=basis_mu, sigma=basis_sigma)
bfr.fit(z_sample[:, None], mu_sample, dmu)
mu_fit = bfr.predict(z[:, None])
mu_sample_fit = bfr.predict(z_sample[:, None])
chi2_dof = (np.sum(((mu_sample_fit - mu_sample) / dmu) ** 2) / (len(mu_sample) - n_constraints))

#------------------------------------------------------------
# Plot the results
fig = plt.figure(figsize=(6, 6))

ax = fig.add_subplot(111)

ax.plot(z, mu_fit, '-k')
ax.plot(z, mu_true, '--', c='gray')
ax.errorbar(z_sample, mu_sample, dmu, fmt='.k', ecolor='gray', lw=1)

ax.text(0.5, 0.05, r"$\chi^2_{\rm dof} = %.2f$" % chi2_dof,
    ha='center', va='bottom', transform=ax.transAxes, fontsize=14)

ax.set_xlim(0.01, 1.8)
ax.set_ylim(36.01, 48)
ax.text(0.05, 0.95, 'Basis Function regression', ha='left', va='top',
                transform=ax.transAxes)

ax.set_ylabel(r'$\mu$')
ax.set_xlabel(r'$z$')

#ax.plot(z, mu_out, '-k', color='red')
plt.show()

Let's now look at the underlying Gaussians that are adding together in the model.

In [ ]:
# Do it by hand so that we can overplot the Gaussians

def gaussian_basis(x, mu, sigma):
    return np.exp(-0.5 * ((x - mu) / sigma) ** 2)

#------------------------------------------------------------
M = np.zeros(shape=[nGaussians, z_sample.shape[0]])
for i in range(nGaussians):
    M[i] = gaussian_basis(z_sample, basis_mu[i], basis_sigma)

M = np.matrix(M).T
C = np.matrix(np.diagflat(dmu**2))
Y = np.matrix(mu_sample).T
coeff = (M.T * C.I * M).I * (M.T * C.I * Y)

#------------------------------------------------------------
# Plot the results
fig = plt.figure(figsize=(6, 6))
fig.subplots_adjust(left=0.1, right=0.95, bottom=0.1, 
                    top=0.95, hspace=0.05, wspace=0.05)

ax = fig.add_subplot(111)

# Plot the gaussians and their sum
i=0
mu_fit = np.zeros(len(z))   
for i in range(nGaussians):
    mu_fit += coeff[i,0] * gaussian_basis(z, basis_mu[i], basis_sigma)
    if (coeff[i,0] > 0.):
        ax.plot(z,coeff[i,0] * gaussian_basis(z, basis_mu[i], basis_sigma), 
                color='C1')
    else:
        ax.plot(z,-coeff[i,0] * gaussian_basis(z, basis_mu[i], basis_sigma), 
                color='C1',ls='--')

#plot the data
ax.plot(z, mu_fit, '-k')
ax.plot(z, mu_true, '--', c='gray')
ax.errorbar(z_sample, mu_sample, dmu, 
            fmt='.k', ecolor='gray', lw=1)

ax.text(0.5, 0.05, r"$\chi^2_{\rm dof} = %.2f$" % chi2_dof,
    ha='center', va='bottom', 
        transform=ax.transAxes, fontsize=14)

ax.set_xlim(0.01, 1.8)
ax.set_ylim(0.01, 48)
ax.text(0.05, 0.95, 'Basis Function regression', ha='left', va='top',
                transform=ax.transAxes)

ax.set_ylabel(r'$\mu$')
ax.set_xlabel(r'$z$')

#ax.plot(z, mu_out, '-k', color='red')
plt.show()

## Kernel Regression <a class="anchor" id="six"></a>

In the case of Gaussian Basis Regression, if you think about it, we were back to the old problem of making a histogram.  Specifically, our Gaussians were evenly spaced over the range of interest. If we instead placed Gaussians at the location of every data point, we get Gaussian Kernel Regression instead. Or just **[Kernel Regression](https://en.wikipedia.org/wiki/Kernel_regression)** more generally since we don't *have* to have a Gaussian kernel function. It is also called **Nadaraya-Watson regression**.

Given a kernel $K(x_i,x)$ (e.g., a Gaussian or top-hat) at each point we estimate the function value by

$$f(x|K) = \frac{\sum_{i=1}^N K\left( \frac{||x_i-x||}{h} \right) y_i}
{\sum_{i=1}^N K\left( \frac{||x_i-x||}{h} \right)},$$

which is a weighted sum of $y$ (weighted by distance) with

$$w_i(x) = \frac{ K\left( \frac{||x_i-x||}{h} \right)}
{\sum_{i=1}^N K\left( \frac{||x_i-x||}{h} \right)}.$$

This **locally weighted regression** technique drives the regressed value to the nearest neighbor (when we have few points) which helps with extrapolation issues. As we saw with KDE, defining the correct bandwidth of the kernel is more important than the shape of the kernel itself and is done through **cross-validation**, which we'll talk about next time.

Nadaraya-Watson is implemented in `AstroML` as follows:

In [ ]:
from astroML.linear_model import NadarayaWatson

Xtest = np.random.random((100,2))
ytest = Xtest[:,0] + Xtest[:,1]

model = NadarayaWatson(kernel='gaussian', h=0.05)
model.fit(Xtest,ytest)

y_pred = model.predict(Xtest)

Using Nadaraya-Watson on our supernova data looks like this:

In [ ]:
from scipy.stats import lognorm
from astroML.datasets import generate_mu_z
from astroML.linear_model import NadarayaWatson

#------------------------------------------------------------
# Generate data
z_sample, mu_sample, dmu = generate_mu_z(100, random_state=0)

from astropy.cosmology import LambdaCDM
cosmo = LambdaCDM(H0=70, Om0=0.30, Ode0=0.70, Tcmb0=0)
z = np.linspace(0.01, 2, 1000)
mu_true = cosmo.distmod(z)

n_constraints = 1

#------------------------------------------------------------
# Plot the results
fig = plt.figure(figsize=(6, 6))

ax = fig.add_subplot(111)

#fit data using standard package
nwreg = NadarayaWatson('gaussian', 0.05)
nwreg.fit(z_sample[:, None], mu_sample)
mu_sample_fit = nwreg.predict(z_sample[:, None])
mu_fit = nwreg.predict(z[:, None])
chi2_dof = (np.sum(((mu_sample_fit - mu_sample) / dmu) ** 2) / 
            (len(mu_sample) - n_constraints))


#plot the data
ax.plot(z, mu_fit, '-k')
ax.plot(z, mu_true, '--', c='gray')
ax.errorbar(z_sample, mu_sample, dmu, fmt='.k', ecolor='gray', lw=1)

ax.text(0.5, 0.05, r"$\chi^2_{\rm dof} = %.2f$" % chi2_dof,
    ha='center', va='bottom', transform=ax.transAxes, fontsize=14)

ax.set_xlim(0.01, 1.8)
ax.set_ylim(36.01, 48)
ax.text(0.05, 0.95, 'Nadaraya-Watson', ha='left', va='top',
                transform=ax.transAxes)

ax.set_ylabel(r'$\mu$')
ax.set_xlabel(r'$z$')

#ax.plot(z, mu_out, '-k', color='red')
plt.show()

## Over/Under-fitting <a class="anchor" id="seven"></a>

We already talked a little bit about overfitting, but let's dive down deeper now that we are trying to fit complicated models. We'll use a 1-D model with homoscedastic errors for the sake of illustration, but this discussion applies to more complicated data as well.

To be clear, our data consists of $X_{\rm train}$, $y_{\rm train}$, and $X_{\rm test}$ and we are trying to predict $y_{\rm test}$.  

Let's take an example where

$$0\le x_i \le 3$$

and

$$y_i = x_i \sin(x_i) + \epsilon_i,$$

where the noise, $\epsilon_i$ is given by $\mathscr{N}(0,0.1)$.

In the example below, we draw 20 evenly spaced points from this distribution and fit them with a straight line.

In [ ]:
# Ivezic v2, Figure 8.12, plot made bigger by SRT and GTR
# Author: Jake VanderPlas
# License: BSD
#   The figure produced by this code is published in the textbook
#   "Statistics, Data Mining, and Machine Learning in Astronomy" (2013)
#   For more information, see http://astroML.github.com
#   To report a bug or issue, use the following forum:
#    https://groups.google.com/forum/#!forum/astroml-general
from matplotlib import ticker
from matplotlib.patches import FancyArrow

#----------------------------------------------------------------------
# This function adjusts matplotlib settings for a uniform feel in the textbook.
# Note that with usetex=True, fonts are rendered with LaTeX.  This may
# result in an error if LaTeX is not installed on your system.  In that case,
# you can set usetex to False.
if "setup_text_plots" not in globals():
    from astroML.plotting import setup_text_plots
setup_text_plots(fontsize=8, usetex=False)

#------------------------------------------------------------
# Define our functional form
def func(x, dy=0.1):
    return np.random.normal(np.sin(x) * x, dy)

#------------------------------------------------------------
# select the (noisy) data
np.random.seed(0)
x = np.linspace(0, 3, 22)[1:-1]
dy = 0.1
y = func(x, dy)

#------------------------------------------------------------
# Select the cross-validation points
np.random.seed(1)
x_cv = 3 * np.random.random(20)
y_cv = func(x_cv)

x_fit = np.linspace(0, 3, 1000)

#------------------------------------------------------------
# First figure: plot points with a linear fit
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)

ax.scatter(x, y, marker='x', c='k', s=30)

p = np.polyfit(x, y, 1)
y_fit = np.polyval(p, x_fit)

ax.text(0.03, 0.96, "d = 1", transform=plt.gca().transAxes,
        ha='left', va='top',
        bbox=dict(ec='k', fc='w'))

ax.plot(x_fit, y_fit, '-b')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')

plt.show()

- This model **underfits** the data and is said to be "biased" (in the sense that the estimated model parameters deviate significantly from the true model parameters).  
- A straight line is a polynomial of order 1, so let's try polynomials of higher order.

In [ ]:
# Ivezic v2, Figure 8.13, panels made bigger by GTR and sizing altered by SRT
# Author: Jake VanderPlas
# License: BSD
#   The figure produced by this code is published in the textbook
#   "Statistics, Data Mining, and Machine Learning in Astronomy" (2013)
#   For more information, see http://astroML.github.com
#   To report a bug or issue, use the following forum:
#    https://groups.google.com/forum/#!forum/astroml-general
from matplotlib import ticker
from matplotlib.patches import FancyArrow

#----------------------------------------------------------------------
# This function adjusts matplotlib settings for a uniform feel in the textbook.
# Note that with usetex=True, fonts are rendered with LaTeX.  This may
# result in an error if LaTeX is not installed on your system.  In that case,
# you can set usetex to False.
if "setup_text_plots" not in globals():
    from astroML.plotting import setup_text_plots
setup_text_plots(fontsize=8, usetex=False)

#------------------------------------------------------------
# Define our functional form
def func(x, dy=0.1):
    return np.random.normal(np.sin(x) * x, dy)

#------------------------------------------------------------
# select the (noisy) data
np.random.seed(0)
x = np.linspace(0, 3, 22)[1:-1]
dy = 0.1
y = func(x, dy)

#------------------------------------------------------------
# Select the cross-validation points
np.random.seed(1)
x_cv = 3 * np.random.random(20)
y_cv = func(x_cv)

x_fit = np.linspace(0, 3, 1000)

#------------------------------------------------------------
# Second figure: plot fit for several orders of polynomial
fig = plt.figure(figsize=(16, 4))
fig.subplots_adjust(wspace=0.03, bottom=0.15,
                    top=0.95, left=0.07, right=0.97)

for i, d in enumerate([2, 3, 19]):  #Try 2nd, 3rd and 19th order
    ax = fig.add_subplot(131 + i)
    ax.scatter(x, y, marker='x', c='k', s=30)

    p = np.polyfit(x, y, d)
    y_fit = np.polyval(p, x_fit)

    ax.plot(x_fit, y_fit, '-b')
    ax.set_ylim(-0.1, 2.1)
    ax.set_xlim(-0.2, 3.2)
    if i in (1, 2):
        ax.yaxis.set_major_formatter(plt.NullFormatter())
    else:
        ax.set_ylabel('$y$')
    ax.set_xlabel('$x$')
    ax.text(0.08, 0.94, "d = %i" % d, transform=ax.transAxes,
            ha='left', va='top',
            bbox=dict(ec='k', fc='w'))

plt.show()

Specifically, $2$, $3$, and all the way up to $19$. We see that $3$ is pretty good in that it seems to be relatively unbiased and also lacks the high variance of the `degree=19` fit (which is also unbiased, but overfit). ***So, how do we choose the "right" answer?***

## Cross-validation <a class="anchor" id="eight"></a>

More regression coefficients improve the ability of the model to fit all the points (reduced **bias**), but at the expense of model complexity and **variance**. *Of course* we can fit a Nth-degree polynomial to N data points, but that would be foolish. We'll determine the best trade-off between bias and variance through [cross-validation](https://en.wikipedia.org/wiki/Cross-validation).

When we increase the complexity of a model, the data points fit the model more and more closely. However, this process does not necessarily result in a better fit to the data. Rather, if the degree is too high, then we are ***overfitting*** the data. The model has high variance, meaning that a small change in a training point can change the model dramatically.  

We can evaluate this using a **training set** ($50-70\%$ of sample), a **cross-validation set** ($15-25\%$) and a **test set** ($15-25\%$).

The training set is used the determine the model paramters, $\theta_j$.  The training data and cross-validation data then are both used to evaluate the training and cross-validation rms errors ($\epsilon_{\rm tr}$ and $\epsilon_{\rm CV}$; evaluated here for polynomial regression):

$$\epsilon_{\rm cv/tr} = \sqrt{\frac{1}{n}\sum_{i=1}^{N_{\rm cv/tr}}
  \left[y_i - \sum_{m=0}^d \theta_0^{(n)}x_i^m\right]}$$

> *Why do we need both a training set and a cross-validation set?* 
> - The **model parameters, $\theta_j$, are learned from the training set**,
> - But the **"hyperparameters" (in this case the model degree) are learned from the cross-validation set**. 

> *The test set then provides the best estimate of the error expected for a new set of unlabeled data.*

We show this graphically in the next figure (Ivezic, 8.14), where the **training and cross-validation rms errors are computed as a function of polynomial degree**, and also compared with the **model AIC and BIC**. 

In [ ]:
# Ivezic v2, Figure 8.14, panels made bigger by GTR
# Author: Jake VanderPlas
# License: BSD
#   The figure produced by this code is published in the textbook
#   "Statistics, Data Mining, and Machine Learning in Astronomy" (2013)
#   For more information, see http://astroML.github.com
#   To report a bug or issue, use the following forum:
#    https://groups.google.com/forum/#!forum/astroml-general
#----------------------------------------------------------------------
# This function adjusts matplotlib settings for a uniform feel in the textbook.
# Note that with usetex=True, fonts are rendered with LaTeX.  This may
# result in an error if LaTeX is not installed on your system.  In that case,
# you can set usetex to False.
if "setup_text_plots" not in globals():
    from astroML.plotting import setup_text_plots
setup_text_plots(fontsize=8, usetex=False)

#------------------------------------------------------------
# Define our functional form
def func(x, dy=0.1):
    return np.random.normal(np.sin(x) * x, dy)

#------------------------------------------------------------
# select the (noisy) data
np.random.seed(0)
x = np.linspace(0, 3, 22)[1:-1]
dy = 0.1
y = func(x, dy)

#------------------------------------------------------------
# Select the cross-validation points
np.random.seed(1)
x_cv = 3 * np.random.random(20)
y_cv = func(x_cv)

x_fit = np.linspace(0, 3, 1000)

#------------------------------------------------------------
# Third figure: plot errors as a function of polynomial degree d
d = np.arange(0, 21)
training_err = np.zeros(d.shape)
crossval_err = np.zeros(d.shape)

fig = plt.figure(figsize=(8, 8))
for i in range(len(d)):
    p = np.polyfit(x, y, d[i])
    training_err[i] = np.sqrt(np.sum((np.polyval(p, x) - y) ** 2)
                              / len(y))
    crossval_err[i] = np.sqrt(np.sum((np.polyval(p, x_cv) - y_cv) ** 2)
                              / len(y_cv))

BIC_train = np.sqrt(len(y)) * training_err / dy + d * np.log(len(y))
BIC_crossval = np.sqrt(len(y)) * crossval_err / dy + d * np.log(len(y))

ax = fig.add_subplot(211)
ax.plot(d, crossval_err, '--k', label='cross-validation')
ax.plot(d, training_err, '-k', label='training')
ax.plot(d, 0.1 * np.ones(d.shape), ':k')

ax.set_xlim(0, 14)
ax.set_ylim(0, 0.8)

ax.set_xlabel('polynomial degree')
ax.set_ylabel('rms error')
ax.legend(loc=2)

ax = fig.add_subplot(212)
ax.plot(d, BIC_crossval, '--k', label='cross-validation')
ax.plot(d, BIC_train, '-k', label='training')

ax.set_xlim(0, 14)
ax.set_ylim(0, 100)

ax.legend(loc=2)
ax.set_xlabel('polynomial degree')
ax.set_ylabel('BIC')

plt.show()

- For low order, both the training and CV error are high. This is sign of a **high-bias model that is underfitting** the data.  
- For high order, the training error becomes small (by definition), but the CV error is large. This is the sign of a **high-variance model that is overfitting** the data. It is matching the subtle fluctuations in the training data that aren't really real, and this shows up in the CV analysis.
- The AIC and BIC give similar results.

Hopefully that helps you understand how to use cross validation to help you both **fit your model and decide on the optimal level of model complexity**, but maybe it doesn't help you apply it to your own data.  

However, we've already seen how to do this with **[GridSearchCV](https://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html#sklearn.grid_search.GridSearchCV)**, where for this case, we'd just be varying one parameter (even though `GridSearchCV` can vary many).

### Learning Curves

We can use a tool called a **[learning curve](https://en.wikipedia.org/wiki/Learning_curve)** to determine if (for a given model) having more training data would help improve the model fitting. This is a different question than above-- rather than try to get a better model of the data, we're trying to improve the quality of our data set. 

The training and CV error are computed as a function of the number of training points. In general:
- ***The training error increases with $N_\mathrm{train}$.*** For a given model, it's easier to fit fewer data points.
- ***The CV error decreases wtih $N_\mathrm{train}$.*** For a given model, a greater number of training points reduces the chances of over-fitting, resulting in better performance of the model in the cross-validation stage.

Let's look at this for the same data and model as above.

In [ ]:
# Ivezic v2, Figure 8.15, panels made bigger by GTR and starting at 2 points.
# Author: Jake VanderPlas
# License: BSD
#   The figure produced by this code is published in the textbook
#   "Statistics, Data Mining, and Machine Learning in Astronomy" (2013)
#   For more information, see http://astroML.github.com
#   To report a bug or issue, use the following forum:
#    https://groups.google.com/forum/#!forum/astroml-general
#----------------------------------------------------------------------
# This function adjusts matplotlib settings for a uniform feel in the textbook.
# Note that with usetex=True, fonts are rendered with LaTeX.  This may
# result in an error if LaTeX is not installed on your system.  In that case,
# you can set usetex to False.
if "setup_text_plots" not in globals():
    from astroML.plotting import setup_text_plots
setup_text_plots(fontsize=8, usetex=True)

#------------------------------------------------------------
# Define our functional form
def func(x, dy=0.1):
    return np.random.normal(np.sin(x) * x, dy)

#------------------------------------------------------------
# select the (noisy) data
np.random.seed(0)
x = np.linspace(0, 3, 22)[1:-1]
dy = 0.1
y = func(x, dy)

#------------------------------------------------------------
# Select the cross-validation points
np.random.seed(1)
x_cv = 3 * np.random.random(20)
y_cv = func(x_cv)

x_fit = np.linspace(0, 3, 1000)

#------------------------------------------------------------
# Fourth figure: plot errors as a function of training set size
np.random.seed(0)
x = 3 * np.random.random(100)
y = func(x)

np.random.seed(1)
x_cv = 3 * np.random.random(100)
y_cv = func(x_cv)

Nrange = np.arange(2, 101, 2)

fig = plt.figure(figsize=(8, 8))
fig.subplots_adjust(left=0.15, top=0.95)

for subplot, d in zip([211, 212], [2, 3]):
    ax = fig.add_subplot(subplot)
    training_err = np.zeros(Nrange.shape)
    crossval_err = np.zeros(Nrange.shape)

    for j, N in enumerate(Nrange):
        p = np.polyfit(x[:N], y[:N], d)
        training_err[j] = np.sqrt(np.sum((np.polyval(p, x[:N])
                                          - y[:N]) ** 2) / len(y))
        crossval_err[j] = np.sqrt(np.sum((np.polyval(p, x_cv)
                                          - y_cv) ** 2) / len(y_cv))

    ax.plot(Nrange, crossval_err, '--k', label='cross-validation')
    ax.plot(Nrange, training_err, '-k', label='training')
    ax.plot(Nrange, 0.1 * np.ones(Nrange.shape), ':k')
    ax.legend(loc=1)
    ax.text(0.03, 0.94, "d = %i" % d, transform=ax.transAxes,
            ha='left', va='top', bbox=dict(ec='k', fc='w'))

    ax.set_ylim(0, 0.4)

    ax.set_xlabel('Number of training points')
    ax.set_ylabel('rms error')

plt.show()

We can see two regimes:

1. ***The training and CV errors have converged.*** This indicates that the model is dominated by bias. Increasing the number of training points is futile. If the error is too high, you instead need a more complex model, not more training data.
2. ***The training error is smaller than the CV error.*** This indicates that the model is dominated by variance.  Increasing the number of training points may help to improve the model.

In both cases, for small numbers of training points, the difference between the training and CV errors indicates that more data well help.  

For the top plot, the convergence of the training and CV errors indicates that adding further data will not reduce the error as it is dominated by ***bias***. A more sophisticated model is needed. The training error starts at zero because we are predicting the training data perfectly, but the CV error is high, indicating that the training data aren't sufficiently representative.

Again, hopefully that helps you understand what learning curves tell you, but maybe it doesn't help you apply it to your own data.  So, let's see how sklearn does it using [sklearn.model_selection.learning_curve](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.learning_curve.html#sklearn.model_selection.learning_curve).  

Below we apply this to the [Boston Housing data](http://scikit-learn.org/stable/datasets/index.html#boston-house-prices-dataset). This data set contains 12 attributes that can be used to predict the price of houses in Boston. 

In [ ]:
#Execute this cell to read in the data
#Also identify the index of the "Number of Rooms" attribute
from sklearn.datasets import load_boston
boston = load_boston()
print(boston.feature_names)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import learning_curve

boston = load_boston()
X = boston.data
y = boston.target

scaler = StandardScaler()
Xscaled = scaler.fit_transform(X)

lin_reg = LinearRegression()
# Ten training sample sizes from 10% to 100%
train_sizes, train_scores_linreg, test_scores_linreg = \
    learning_curve(lin_reg, Xscaled, y, train_sizes=np.linspace(0.1, 1, 10), \
    scoring="neg_mean_squared_error", cv=10)

fig = plt.figure(figsize=(8, 8))

plt.plot(train_sizes, -test_scores_linreg.mean(1), 'o-', color="r", label="Val")
plt.plot(train_sizes, -train_scores_linreg.mean(1), 'o-', color="g", label="Train")
                   
plt.xlabel("Train size",fontsize=14)
plt.ylabel("Mean Squared Error",fontsize=14)
plt.title('Learning curves',fontsize=14)
plt.legend(loc="best")
plt.ylim(0,200)

#plt.show()

So, we barely have enough data.  This is important to know when we are deciding on the number of cross validations.  

Once you are happy with the size of your training set and your choice of model, and now just want to get the best model parameters using *all* of the data (not just whatever fraction is in your training set), then we can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)**.

Let's use this to predict the price of houses in Boston where each house gets to be part of one of the test sets by doing a $10$-fold cross validation. We will then plot the *prediction* versus the *actual* value of $y$, which should lie along the line $y=y_\mathrm{pred}$.

In [ ]:
from sklearn.model_selection import cross_val_predict

scaler = StandardScaler() #Complete
Xscaled = scaler.fit_transform(X) #Complete
linreg = LinearRegression() #Complete
linreg.fit(Xscaled, y) #Complete

# Do a 10-fold cross validation
# Then try a 3-fold cross validation
# Can you understand the difference?
ypred = cross_val_predict(linreg, Xscaled, y, cv=10)

fig, ax = plt.subplots(figsize=(8,8))
ax.scatter(y, ypred, edgecolors=(0, 0, 0))
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Actual [x1000]',fontsize=14)
ax.set_ylabel('Predicted [x1000]',fontsize=14)
plt.show()

<font color='red'>Now try a 3-fold cross validation. Can you understand the difference?</font>

As a final recap, in case you're still not sure about $k$-fold CV, the procedure is thus:

- Split the data into $k+1$ subsets; the test set and $k$ CV sets. How you do this is up to you, but typically through random shufflings with equal numbers of points.
- $k$ models are trained, each time leaving out one of the CV sets in order to measure the CV error.
- The final training and CV error can be computed using the mean or median of the set of results. The median can be more reliable.

See the following GIF of $3$-fold CV from the [Wikipedia article](https://www.wikiwand.com/en/Cross-validation_(statistics)).


![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4b/KfoldCV.gif/1920px-KfoldCV.gif?1616979144969)